# IXS/ETH Vault Performance Analysis

**Deliverable 2:** Track the Arrakis-managed V4 vault since inception. Compare against HODL and full-range LP benchmarks.

**Vault:** `0x90bde935ce7feb6636afd5a1a0340af45eeae600`
- token0 = IXS, token1 = Native ETH (0xEeee...EEeE)
- Manages a concentrated liquidity position in the IXS/ETH V4 pool

In [ ]:
import os
import sys
import math
from web3 import Web3
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("..")

from src.config import (
    STATE_VIEW, IXS, ETH,
    IXS_POOL_ID_BYTES, ARRAKIS_VAULT, CHAINLINK_ETH_USD,
)
from src.abis import STATEVIEW_ABI_EXTENDED, ARRAKIS_VAULT_ABI, CHAINLINK_ABI
from src.block_utils import (
    generate_daily_block_samples, blocks_to_timestamps,
    timestamps_to_dates, get_latest_block,
)
from src.price_feeds import (
    get_eth_usd_at_block, get_ixs_eth_price_from_v4,
    get_ixs_eth_price_from_v2,
)
from src.vault_performance import (
    batch_vault_underlying,
    calculate_hodl_value, calculate_fullrange_lp_value,
)
from src.migration_detection import find_v4_pool_creation_block

load_dotenv()
w3 = Web3(Web3.HTTPProvider(os.getenv('rpc_url_mainnet')))
print(f'Connected: {w3.is_connected()}')
print(f'Latest block: {w3.eth.block_number}')

In [ ]:
stateview = w3.eth.contract(address=Web3.to_checksum_address(STATE_VIEW), abi=STATEVIEW_ABI_EXTENDED)
vault = w3.eth.contract(address=Web3.to_checksum_address(ARRAKIS_VAULT), abi=ARRAKIS_VAULT_ABI)
chainlink = w3.eth.contract(address=Web3.to_checksum_address(CHAINLINK_ETH_USD), abi=CHAINLINK_ABI)

# Vault token0 = IXS, token1 = native ETH
vt0 = vault.functions.token0().call()
vt1 = vault.functions.token1().call()
ixs_is_token0_vault = vt0.lower() == IXS.lower()
print(f"Vault token0: {vt0} ({'IXS' if ixs_is_token0_vault else 'ETH'})")
print(f"Vault token1: {vt1} ({'ETH' if ixs_is_token0_vault else 'IXS'})")

## 1. Vault Start & Daily Sampling

In [ ]:
latest_block = get_latest_block(w3)
search_start = latest_block - 7200 * 365

# Use V4 pool creation as start (need pool prices to exist)
print("Finding V4 pool creation block...")
v4_start = find_v4_pool_creation_block(
    stateview, IXS_POOL_ID_BYTES, search_start, latest_block
)
print(f"V4 start: block {v4_start:,}")

sample_blocks = generate_daily_block_samples(v4_start, latest_block)
print(f"Daily samples: {len(sample_blocks)}")

print("Fetching timestamps...")
sample_ts = blocks_to_timestamps(w3, sample_blocks)
sample_dates = timestamps_to_dates(sample_ts)
dates = pd.to_datetime(sample_dates)
print(f"Date range: {sample_dates[0]} to {sample_dates[-1]}")

## 2. Vault Token Amounts Over Time

In [ ]:
print("Fetching vault underlying at each block...")
vault_data = batch_vault_underlying(vault, sample_blocks)

# Parse: vault token0=IXS, token1=ETH
if ixs_is_token0_vault:
    ixs_amounts = [d[1] / 1e18 for d in vault_data]
    eth_amounts = [d[2] / 1e18 for d in vault_data]
else:
    ixs_amounts = [d[2] / 1e18 for d in vault_data]
    eth_amounts = [d[1] / 1e18 for d in vault_data]

print(f"Initial: {ixs_amounts[0]:,.2f} IXS, {eth_amounts[0]:.4f} ETH")
print(f"Current: {ixs_amounts[-1]:,.2f} IXS, {eth_amounts[-1]:.4f} ETH")

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 6))

ax1.set_xlabel('Date')
ax1.set_ylabel('IXS Amount', color='tab:blue')
ax1.plot(dates, ixs_amounts, color='tab:blue', alpha=0.8, linewidth=1.5, label='IXS')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.tick_params(axis='x', rotation=45)

ax2 = ax1.twinx()
ax2.set_ylabel('ETH Amount', color='tab:orange')
ax2.plot(dates, eth_amounts, color='tab:orange', alpha=0.8, linewidth=1.5, label='ETH')
ax2.tick_params(axis='y', labelcolor='tab:orange')

fig.suptitle('Arrakis Vault: Token Amounts Over Time', fontsize=14, fontweight='bold')
fig.legend(loc='upper left', bbox_to_anchor=(0.12, 0.88))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/ixs_vault_token_amounts.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Price Data

- ETH/USD from Chainlink oracle
- IXS/ETH from V4 pool sqrtPriceX96
- IXS/USD derived = IXS/ETH × ETH/USD

In [ ]:
from src.price_feeds import batch_eth_usd_prices

print("Fetching ETH/USD from Chainlink...")
eth_usd_prices = batch_eth_usd_prices(chainlink, sample_blocks)

print("Fetching IXS/ETH from V4 pool...")
ixs_eth_prices = []
for block in sample_blocks:
    try:
        # ETH is currency0 (address(0)), IXS is currency1
        # sqrtPriceX96 gives IXS/ETH (token1 per token0)
        # We want ETH per IXS for pricing
        price = get_ixs_eth_price_from_v4(
            stateview, IXS_POOL_ID_BYTES, block, ixs_is_currency0=False
        )
        ixs_eth_prices.append(price)
    except Exception:
        ixs_eth_prices.append(0.0)

# IXS/USD = (ETH per IXS) × ETH/USD
ixs_usd_prices = [ie * eu for ie, eu in zip(ixs_eth_prices, eth_usd_prices)]

print(f"\nETH/USD range: ${min(eth_usd_prices):,.2f} - ${max(eth_usd_prices):,.2f}")
nz = [p for p in ixs_usd_prices if p > 0]
if nz:
    print(f"IXS/USD range: ${min(nz):.6f} - ${max(nz):.6f}")

## 4. Vault Composition Over Time (USD)

In [ ]:
ixs_usd_values = [amt * price for amt, price in zip(ixs_amounts, ixs_usd_prices)]
eth_usd_values = [amt * price for amt, price in zip(eth_amounts, eth_usd_prices)]
total_usd = [i + e for i, e in zip(ixs_usd_values, eth_usd_values)]

fig, ax = plt.subplots(figsize=(14, 6))
ax.stackplot(dates, ixs_usd_values, eth_usd_values,
             labels=['IXS (USD)', 'ETH (USD)'],
             colors=['#4e79a7', '#f28e2b'], alpha=0.8)
ax.plot(dates, total_usd, 'k-', linewidth=1.5, alpha=0.5, label='Total')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Arrakis Vault: Composition Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_composition.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Initial value: ${total_usd[0]:,.2f}")
print(f"Current value: ${total_usd[-1]:,.2f}")

## 5. Vault vs HODL Benchmark

HODL benchmark: hold the initial token amounts without providing liquidity.

In [ ]:
init_ixs = ixs_amounts[0]
init_eth = eth_amounts[0]

hodl_values = [
    calculate_hodl_value(init_ixs, init_eth, iu, eu)
    for iu, eu in zip(ixs_usd_prices, eth_usd_prices)
]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, total_usd, 'g-', linewidth=2, label='Arrakis Vault')
ax.plot(dates, hodl_values, 'b--', linewidth=2, label='HODL')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Vault vs HODL', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_vs_hodl.png', dpi=150, bbox_inches='tight')
plt.show()

if hodl_values[-1] > 0:
    print(f"Vault: ${total_usd[-1]:,.2f}")
    print(f"HODL:  ${hodl_values[-1]:,.2f}")
    print(f"Diff:  {(total_usd[-1]/hodl_values[-1] - 1)*100:+.2f}%")

## 6. Full-Range LP Benchmark

Full-range LP follows the V2 impermanent loss formula:

$$V_{LP} = V_{HODL} \times \frac{2\sqrt{r}}{1 + r}$$

where $r$ is the relative price ratio change between the two assets.

In [ ]:
init_ixs_usd = ixs_usd_prices[0]
init_eth_usd = eth_usd_prices[0]

fullrange_values = [
    calculate_fullrange_lp_value(
        init_ixs, init_eth, init_ixs_usd, init_eth_usd, iu, eu
    )
    for iu, eu in zip(ixs_usd_prices, eth_usd_prices)
]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, total_usd, 'g-', linewidth=2, label='Arrakis Vault (Concentrated)')
ax.plot(dates, hodl_values, 'b--', linewidth=2, label='HODL')
ax.plot(dates, fullrange_values, 'r:', linewidth=2, label='Full-Range LP (V2-style)')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Vault vs HODL vs Full-Range LP', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Performance Summary & Tradeoffs

In [ ]:
print("=" * 60)
print("VAULT PERFORMANCE SUMMARY")
print("=" * 60)
print(f"Period: {sample_dates[0]} to {sample_dates[-1]}")
print(f"\nInitial: {init_ixs:,.2f} IXS + {init_eth:.4f} ETH = ${total_usd[0]:,.2f}")

print(f"\nFinal values:")
print(f"  Vault:      ${total_usd[-1]:,.2f}")
print(f"  HODL:       ${hodl_values[-1]:,.2f}")
print(f"  Full-Range: ${fullrange_values[-1]:,.2f}")

if total_usd[0] > 0:
    print(f"\nReturns vs initial:")
    print(f"  Vault:      {(total_usd[-1]/total_usd[0] - 1)*100:+.2f}%")
    print(f"  HODL:       {(hodl_values[-1]/total_usd[0] - 1)*100:+.2f}%")
    print(f"  Full-Range: {(fullrange_values[-1]/total_usd[0] - 1)*100:+.2f}%")

if hodl_values[-1] > 0:
    il_fr = (fullrange_values[-1]/hodl_values[-1] - 1) * 100
    il_vault = (total_usd[-1]/hodl_values[-1] - 1) * 100
    print(f"\nImpermanent Loss (vs HODL):")
    print(f"  Full-Range: {il_fr:+.2f}%")
    print(f"  Vault:      {il_vault:+.2f}%")
    print(f"\nThe vault {'outperforms' if il_vault > il_fr else 'underperforms'} full-range LP by {abs(il_vault - il_fr):.2f}pp")